In [1]:
import pandas as pd
import os

In [2]:
## Create database
from sqlalchemy import create_engine
engine = create_engine('sqlite:///hawaii.sqlite')

# Import and establish Base for which classes will be constructed
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float, ForeignKey, Text


In [3]:
## Define the measurement class
class Measurement(Base):
    __tablename__ = 'measurement'
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(String)
    prcp = Column(Float)
    tobs = Column(Float)
    
    def __repr__(self):
        return f'id={self.id}, name={self.station}'

In [4]:
## Define the stations class
class Station(Base):
    __tablename__ = 'station'
    station = Column(String, ForeignKey('measurement.station'), primary_key=True)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)


In [5]:
## create tables
Base.metadata.create_all(engine)

In [6]:
# Use Pandas to Bulk insert each CSV file into their appropriate table
def populate_table(engine, table, csvfile):
    # connect to the database
    conn = engine.connect()
    # Load the CSV file into a pandas dataframe
    df_of_data_to_insert = pd.read_csv(csvfile)
    # Orient='records' creates a list of data to write
    data = df_of_data_to_insert.to_dict(orient='records')
    # Delete all rows in the table
    conn.execute(table.delete())
    # Insert the dataframe into the database in one bulk insert
    conn.execute(table.insert(), data)


In [7]:
# Call the function to insert the data for each table
## Read clean csv's
clean_hawaii_stations_path = os.path.join('Resources', 'clean_hawaii_stations.csv')
clean_hawaii_measurements_path = os.path.join('Resources', 'clean_hawaii_measurements.csv')
populate_table(engine, Station.__table__, clean_hawaii_stations_path)
populate_table(engine, Measurement.__table__, clean_hawaii_measurements_path)